In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from DNN_module import Net

# Number of data points
n = 3000

# time inteval
delta = 1/52

# Observation time
obtime = np.arange(0,n+1)/n * n * delta

%run ../NCoinDP_functions.ipynb

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu"
device = torch.device(dev) 

2.1.2+cu121
True
NVIDIA A100-PCIE-40GB


In [ ]:
# Number of synthetic data
L = 2500000

# Range of parameters
mu_range = [1, 5]
theta_range = [1, 2.5]
sigma2_range = [0.5, 2]

# Training + validation + Test data generating
torch.manual_seed(515)
mu_ran     = torch.rand(L) * (mu_range[1] -  mu_range[0]) + mu_range[0]
theta_ran  = torch.rand(L) * (theta_range[1] -  theta_range[0]) + theta_range[0]
sigma2_ran = torch.rand(L) * (sigma2_range[1] -  sigma2_range[0]) + sigma2_range[0]

# Output: L x 3 matrix
output = torch.stack((mu_ran, theta_ran, sigma2_ran), dim = 1)

# J Synthetic data generating
torch.manual_seed(515)
y0_tmp = torch.normal(theta_ran, torch.sqrt(sigma2_ran/(2*mu_ran)))
yL = OU_simul_sample(L, obtime, y0_tmp, mu_ran, theta_ran, sigma2_ran ** (1/2))

# Summary Statistics
X = OU_summary(yL)
tmp = "infer_sim_0.pt"
torch.save([X, output], tmp)

In [3]:
import torch
import math
from torch.distributions.gamma import Gamma
torch.set_default_device("cpu")
# Number of synthetic data
L = 2500000
#L = 500000

# hyper-parameters 
mu_par = [2, 2 ** (1/2)]
theta_par = [1, 2 ** (1/2)]
sigma2_par = [1, 1]

# Training + validation + Test data generating
torch.manual_seed(510)
mu_ran = torch.empty(L)
theta_ran = torch.empty(L)
sigma2_ran = torch.empty(L)

mu_ran = torch.nn.init.trunc_normal_(mu_ran, mean = mu_par[0], std = mu_par[1], a = 0, b= math.inf)
theta_ran = torch.nn.init.trunc_normal_(theta_ran, mean = theta_par[0], std = theta_par[1], a = 0, b= math.inf)
sigma2_ran = Gamma(torch.ones(L)*sigma2_par[0], torch.ones(L)*sigma2_par[1]).sample()

# Output: L x 3 matrix
output = torch.stack((mu_ran, theta_ran, sigma2_ran), dim = 1)

# J Synthetic data generating
torch.manual_seed(511)
y0_tmp = torch.normal(theta_ran, torch.sqrt(sigma2_ran/(2*mu_ran)))
yL = OU_simul_sample(L, obtime, y0_tmp, mu_ran, theta_ran, sigma2_ran ** (1/2))

# Summary Statistics
X = OU_summary(yL)
tmp = "infer_sim_1.pt"

torch.save([X, output], tmp)